## Data Warehousing HW 3 - Grocery Store Data Profiling
## Anthony Ung, Sean Jerzewski, Gideon Kipkorir

### 0. Connecting to Database

In [1]:
import sqlite3 as lite
import csv

def GroceryDatabaseAccess():
    GroceryDatabaseAccess.grocerDBConnection
    GroceryDatabaseAccess.groceryDBCursor
    GroceryDatabaseAccess.salesTransactionsToCommitCount
    GroceryDatabaseAccess.maxTransactionsBeforeCommit

def init():
    GroceryDatabaseAccess.groceryDBConnection = None
    GroceryDatabaseAccess.groceryDBCursor = None
    GroceryDatabaseAccess.salesTransactionsToCommitCount = 0   
    GroceryDatabaseAccess.maxTransactionsBeforeCommit = 10000 

def run():
    print("Connecting to the grocerydb database")
    con = lite.connect(r'store.db')
    print('Database successfully connected to')
    GroceryDatabaseAccess.groceryDBConnection = con
    GroceryDatabaseAccess.groceryDBCursor = con.cursor()

def build_products_table():
    sql = 'DROP TABLE IF EXISTS products'
    GroceryDatabaseAccess.groceryDBCursor.execute(sql)
    
    sql = 'CREATE TABLE products(sku INT, product_name TEXT, product_type TEXT)'
    GroceryDatabaseAccess.groceryDBCursor.execute(sql)

    csv.register_dialect('piper', delimiter='|', quoting=csv.QUOTE_NONE)
    
    with open('Products1.txt', 'r') as csvfile:
        i = 0
        
        for row in csv.DictReader(csvfile, dialect='piper'):
            sku = row.get('SKU')
            product_name = row.get('Product Name')
            product_type = row.get('itemType')
            GroceryDatabaseAccess.groceryDBCursor.execute('insert into products values (?, ?, ?)',
                                                      (sku,product_name, product_type))
            i += 1
            if i % 10000 == 0:
                GroceryDatabaseAccess.groceryDBConnection.commit()
                print(f"Committed row {i}")
            
        GroceryDatabaseAccess.groceryDBConnection.commit()
        print(f"Committed row {i}")

run()

Connecting to the grocerydb database
Database successfully connected to


### 0.1. Build Transactions Table

In [3]:
from datetime import date, datetime, timedelta


def test_dates():
    print('''
            This should print the first 5 and last 5 dates.
            I need subqueries because in SQLite, ORDER BY comes after UNION
                and I need subqueries to order by ASC and DESC in the two parts independently.
            In this case, we are missing the dates '2024-01-01' and '2024-12-31'.
        ''')

    
    sql =   '''
                SELECT * FROM
                    (SELECT DISTINCT date
                    FROM sales_transactions
                    ORDER BY date ASC
                    LIMIT 5
                    )

                UNION

                SELECT * FROM
                    (SELECT DISTINCT date
                    FROM sales_transactions
                    ORDER BY date DESC
                    LIMIT 5
                    )
            '''
    results = GroceryDatabaseAccess.groceryDBCursor.execute(sql)
    for row in results:
        print(row)

def build_transactions_table():
    cur = GroceryDatabaseAccess.groceryDBCursor
    
    sql = "DROP TABLE IF EXISTS transactions"
    cur.execute(sql)

    sql = '''
            CREATE TABLE transactions(date VARCHAR(8), customerNumber INT, total FLOAT)
        '''
    cur.execute(sql)

    current_date = date(2024, 1, 1)
    end_date = date(2024, 12, 31)

    
def query():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection
    
    
    current_date = date(2024, 1, 1)
    end_date = date(2024, 12, 31)
    
    num_days = 1
    
    while(current_date <= end_date):
        sql = '''
                SELECT date, customerNumber, ROUND(SUM(salesPrice),2) 
                FROM sales_transactions 
                WHERE date == (?)
                GROUP BY customerNumber
            '''
        num_records = 0
        
        results = cur.execute(sql, (current_date,)).fetchall()
        for row in results:
            num_records += 1
            data = (row[0], row[1], row[2])
            cur.execute('INSERT INTO transactions VALUES (?, ?, ?)', data)
        con.commit()

        if num_days % 30 == 1:
            print(f'{datetime.now()} - Committed transaction results for date {current_date} - {num_records} records committed')

        num_days += 1
        current_date += timedelta(1)
    

def run_0_1():
    #test_dates()
    build_transactions_table()
    query()

run_0_1()

C:\Users\Anthony\AppData\Local\Temp\ipykernel_17380\3616095981.py:68: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  results = cur.execute(sql, (current_date,)).fetchall()


2025-02-17 12:10:07.359552 - Committed transaction results for date 2024-01-01 - 1052 records committed
2025-02-17 12:10:42.735042 - Committed transaction results for date 2024-01-31 - 1044 records committed
2025-02-17 12:11:18.066887 - Committed transaction results for date 2024-03-01 - 1023 records committed
2025-02-17 12:11:53.491544 - Committed transaction results for date 2024-03-31 - 1130 records committed
2025-02-17 12:12:27.989817 - Committed transaction results for date 2024-04-30 - 1042 records committed
2025-02-17 12:13:04.346983 - Committed transaction results for date 2024-05-30 - 1044 records committed
2025-02-17 12:13:38.222962 - Committed transaction results for date 2024-06-29 - 1130 records committed
2025-02-17 12:14:13.450154 - Committed transaction results for date 2024-07-29 - 1029 records committed
2025-02-17 12:14:50.588741 - Committed transaction results for date 2024-08-28 - 1058 records committed
2025-02-17 12:15:25.116392 - Committed transaction results for d

## 0.2 - Various tests

In [4]:
def test_daily_customer_counts():
    cur = GroceryDatabaseAccess.groceryDBCursor

    sql = \
        '''
            WITH customer_counts AS (
                SELECT date, COUNT (DISTINCT customerNumber)
                FROM sales_transactions
                GROUP BY date
            )
        '''


tests = \
    { \
        'Daily Customer Counts' : True
    }

if tests['Daily Customer Counts']:
    test_daily_customer_counts()

## 0.3 - Preliminary Profiling
Given our parameters,
1. There should be about 15,000 transactions
2. There should be about 10,500 transactions involving sales of milk.

In [5]:
def run_0_3():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            SELECT COUNT(*) AS count
            FROM transactions
            WHERE (date >= '2024-01-02') AND (date <= '2024-01-15')
        '''

    results = cur.execute(sql)
    for row in results:
        print(f'{row[0]} transactions in database')
    
    sql = '''
            SELECT COUNT(*) AS count
            FROM sales_transactions
            JOIN products on sales_transactions.sku == products.sku
            WHERE (date >= '2024-01-02') AND (date <= '2024-01-15') AND (product_type == 'Milk')
        '''

    results = cur.execute(sql)
    for row in results:
        print(f'{row[0]} transactions involving milk')

run_0_3()

14791 transactions in database
12161 transactions involving milk


## 1.1 - Average Sales Per Day
Compute the average sales per day of each item

Our parameters
- 1020-1060 Customers Daily + 75 for Weekends
- 1-90 Items Per Customer

Assumptions Given
- About 70% of our customers are supposed to buy milk
  - 35% will buy milk and cereal
  - 1.5% will buy cereal but no milk
- 20% will buy baby food
  - 16% will buy baby food and diapers
  - 0.8% will buy diapers but no baby food
- 50% will buy bread
- 10% will buy peanut butter
  - 9% will buy peanut butter + jam/jelly
  - 4.5% will buy jam + jelly but no peanut butter
- All other products should be equally likely

In [6]:
def run_1_1():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH selected_transacs AS (
                SELECT *
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-16')
            )
            SELECT 
                products.product_name, 
                products.product_type,
                count(*)/14 AS count
            FROM products
            JOIN selected_transacs USING(sku)
            GROUP BY products.sku
            ORDER BY count DESC
            LIMIT 25
        '''

    results = cur.execute(sql)
    for row in results:
        print(row)

run_1_1()

('1.00% Milk', 'Milk', 160)
('2.00% Milk', 'Milk', 158)
('1.00% Milk', 'Milk', 157)
('Whole Milk Milk', 'Milk', 153)
('Whole Milk Milk', 'Milk', 151)
('2.00% Milk', 'Milk', 151)
('Squeeze Jelly Grape', 'Jelly/Jam', 62)
('Jelly Grape', 'Jelly/Jam', 61)
('Jam Strawberry', 'Jelly/Jam', 60)
('Jam Grape', 'Jelly/Jam', 59)
('Farmhouse Bread Potato', 'Bread', 39)
('Ezekiel 4:9 Bread Sesame Sprouted Grain', 'Bread', 38)
('Whole Grain Bread 100% Wheat Bread', 'Bread', 38)
('Flatbread Angus Cheeseburgers', 'Bread', 38)
('Bread Honey Wheat', 'Bread', 38)
('Sandwich Thins Potato Rolls', 'Bread', 37)
('Swirl Oatmeal Bread Apple & Cinnamon', 'Bread', 37)
('Light Style Bread Soft Wheat', 'Bread', 37)
('Italian Bread With Sesame Seeds', 'Bread', 37)
('Farmhouse Bread Oatmeal', 'Bread', 37)
('Bread Swirl Raisin Cinnamon', 'Bread', 37)
('Bread Texas Toast', 'Bread', 37)
('Bread Amazin Raisin', 'Bread', 37)
('Sandwich Thins Whole Wheat', 'Bread', 36)
('Whole Grain Bread Honey Wheat', 'Bread', 36)


## 1.1 - Average Sales Per Item Type Per Day
It may also be valuable to see the numbers of each product type sold per day

In [7]:
def run_1_1a():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH selected_transacs AS (
                SELECT *
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-15')
            )
            SELECT 
                products.product_type,
                count(*)/14 AS count
            FROM products
            JOIN selected_transacs USING(sku)
            GROUP BY products.product_type
            HAVING products.product_type IN ('Milk', 'Cereal', 'Diapers', 'Baby Food', 'Bread', 'Peanut Butter', 'Jelly/Jam')
            ORDER BY count DESC
        '''

    results = cur.execute(sql)
    for row in results:
        print(row)

run_1_1a()

('Baby Food', 3803)
('Cereal', 2440)
('Diapers', 1997)
('Bread', 1607)
('Milk', 868)
('Peanut Butter', 547)
('Jelly/Jam', 229)


## 1.1 - Average Sales Per Item Type Per Transaction Per Day
It may also be valuable to see the numbers of each product type sold per transaction per day

In [8]:
def run_1_1b():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH selected_transacs AS (
                SELECT date, customerNumber, sku
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-16')
            ),
            type_counts_per_transaction AS (
                SELECT date, customerNumber, products.product_type AS type, COUNT(*) as count
                FROM selected_transacs
                JOIN products USING(sku)
                GROUP BY date, customerNumber, products.product_type
            ),
            type_frequencies_per_day AS (
                SELECT type, COUNT(*)/14 AS freq
                FROM type_counts_per_transaction
                GROUP BY type
                HAVING type IN ('Milk', 'Cereal', 'Diapers', 'Baby Food', 'Bread', 'Peanut Butter', 'Jelly/Jam')
                ORDER BY freq DESC
            )
            SELECT * from type_frequencies_per_day
        '''

    results = cur.execute(sql)
    for row in results:
        print(row)

run_1_1b()

('Baby Food', 991)
('Cereal', 950)
('Bread', 892)
('Diapers', 865)
('Milk', 838)
('Peanut Butter', 440)
('Jelly/Jam', 226)


In [ ]:
def run_test():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH selected_transacs AS (
                SELECT date, customerNumber, sku
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-15')
            ),
            type_counts_per_transaction AS (
                SELECT date, customerNumber, products.product_type AS type, COUNT(*) as count
                FROM selected_transacs
                JOIN products USING(sku)
                GROUP BY date, customerNumber, products.product_type
            ),
            type_frequencies_per_day AS (
                SELECT type, COUNT(*)/14 AS freq
                FROM type_counts_per_transaction
                GROUP BY type
                HAVING type IN ('Milk', 'Cereal', 'Diapers', 'Baby Food', 'Bread', 'Peanut Butter', 'Jelly/Jam')
                ORDER BY freq DESC
            )
            SELECT * from type_frequencies_per_day
        '''

    results = cur.execute(sql)
    for row in results:
        print(row)

run_test()

## Average number of customer numbers per day

In [17]:
def run_test():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH customers_per_day AS (
                SELECT date, MAX(customerNumber) as numCustomers
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-15')
                GROUP BY date
            )
            SELECT AVG(numCustomers)
            FROM customers_per_day
        '''

    results = cur.execute(sql)
    for row in results:
        print(row)

run_test()

(1056.5,)


## Average Number of Items Sold Per Day

In [20]:
def run_test():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH customers_per_day AS (
                SELECT date, COUNT(*) as itemsSold
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-15')
                GROUP BY date
            )
            SELECT ROUND(AVG(itemsSold), 0)
            FROM customers_per_day
        '''

    results = cur.execute(sql)
    for row in results:
        print(row)

run_test()

(48094.0,)


## Minimum and Maximum for non-special items

In [33]:
def run_test():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH items_sold AS (
                SELECT sku
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-15')
            ),
            items_sold_count AS (
                SELECT sku, COUNT(*) as count
                FROM items_sold
                GROUP BY sku
            ),
            filtered_items_list AS (
                SELECT isc.sku, isc.count AS count, p.product_type
                FROM items_sold_count AS isc
                JOIN products AS p USING(sku)
                WHERE p.product_type NOT IN ('Milk', 'Cereal', 'Diapers', 'Baby Food', 'Bread', 'Peanut Butter', 'Jelly/Jam')
            )
            SELECT MIN(fil.count), MAX(fil.count) FROM filtered_items_list AS fil
        '''
    
    results = cur.execute(sql)
    for row in results:
        print(row)

run_test()

(252, 366)


In [40]:
def run_test():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            SELECT p.product_type, COUNT(*)
            FROM products AS p
            WHERE p.product_type IN ('Milk', 'Cereal', 'Diapers', 'Baby Food', 'Bread', 'Peanut Butter', 'Jelly/Jam')
            GROUP BY p.product_type
        '''
    
    results = cur.execute(sql)
    for row in results:
        print(row)

run_test()

('Baby Food', 162)
('Bread', 48)
('Cereal', 93)
('Diapers', 82)
('Jelly/Jam', 4)
('Milk', 6)
('Peanut Butter', 20)


In [42]:
def run_test():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            SELECT DISTINCT p.product_type
            FROM products AS p
            ORDER BY 1 ASC
        '''
    
    results = cur.execute(sql)
    for row in results:
        print(row)

run_test()

('',)
('Acetominifen',)
('Aspirin',)
('Baby Food',)
('Baked Goods Other than Bread',)
('Baking Supplies',)
('Bologna',)
('Bread',)
('Cake Snacks',)
('Cake/Baking Mixes',)
('Candy',)
('Canned Fruit',)
('Canned Goods',)
('Canned Vegetables',)
('Cereal',)
('Cheese',)
('Chocolate Candy',)
('Cleaners',)
('Coffee/Creamer',)
('Cookies',)
('Cooking Oil',)
('Cottage Cheese',)
('Deli Meats',)
('Deli Salads',)
('Deodorizers',)
('Diapers',)
('Dips',)
('Drink',)
('French Fries',)
('Fresh Chicken',)
('Fresh Fish',)
('Fresh Fruit',)
('Fresh Vegetables',)
('Frozen Chicken',)
('Frozen Food',)
('Frozen Vegetables',)
('Gravy/Sauce',)
('Hamburger',)
('Hard Candy',)
('Hot Dogs',)
('Household',)
('Ibuprofen',)
('Ice Cream',)
('Jelly/Jam',)
('Juice',)
('Mac & Cheese',)
('Milk',)
('Mouthwash',)
('Nuts',)
('Other Dairy/Not Milk',)
('Pasta/Noodles',)
('Peanut Butter',)
('Pet Food',)
('Pizza',)
('Popcorn',)
('Popsicles',)
('Produce',)
('Rice',)
('Rice/Rice Mix',)
('Salad Dressing',)
('Sardines',)
('Snacks',)
('S

In [44]:
def run_test():
    cur = GroceryDatabaseAccess.groceryDBCursor
    con = GroceryDatabaseAccess.groceryDBConnection

    sql = '''
            WITH items_sold AS (
                SELECT sku
                FROM sales_transactions
                WHERE (date >= '2024-01-02') AND (date <= '2024-01-15')
            )
            SELECT COUNT(*) FROM items_sold
        '''
    
    results = cur.execute(sql)
    for row in results:
        print(row)

run_test()

(673319,)
